In [2]:
import pandas as pd
import numpy as np

This is from Weber et al.
https://doi.org/10.1016/j.immuni.2021.12.003

- Supplement has neutralization and EC50 values, but the names are different in the plabsssssssDab
- 92/310 have poor EC50 (>1 ug/mL)
- All have at least some neutralization %

They initially screened serum, then calculated neutralization across multiple strains
- 92% neutralized against genotype 5a, so I am using that here

In [28]:
imgt_seqs = pd.read_csv('./weber_HCV_seqs_v2_IMGT/5_AA-sequences.txt', sep='\t', index_col=0)
imgt_seqs['Sequence ID'] = imgt_seqs['Sequence ID'].apply(lambda x: x.split('|')[1].split('.')[0])
# imgt_seqs.index = imgt_seqs['Sequence ID']

In [17]:
clone_id_mapping = pd.read_csv('sequences_GenBank_clone_headers_weber_HCV.fasta', sep='\t')

In [19]:
from Bio import SeqIO

headers = []
with open('sequences_GenBank_clone_headers_weber_HCV.fasta', "r") as f:
    for record in SeqIO.parse(f, "fasta"):
        headers.append(record.description)

In [26]:
genbank_ids = [i.split('.')[0] for i in headers]
clone_names = [i.split('clone ')[1].split(' ')[0] for i in headers]

conversion_dict = dict(zip(genbank_ids, clone_names))

In [32]:
imgt_seqs['clone_name'] = imgt_seqs['Sequence ID'].map(conversion_dict)
imgt_seqs = imgt_seqs[~imgt_seqs['clone_name'].isna()]
imgt_seqs.index = imgt_seqs['clone_name']

In [57]:
imgt_hc = imgt_seqs[~imgt_seqs['V-D-J-REGION'].isna()]
imgt_lc = imgt_seqs[imgt_seqs['V-D-J-REGION'].isna()]

In [46]:
binding_table = pd.read_excel('webb_supp2.xlsx', index_col=0)
binding_table['name'] = [i[5:] for i in binding_table.index]

In [59]:
# Mapping in sequences from GenBank
binding_table['VH_AA'] = binding_table['name'].map(dict(zip(imgt_hc['clone_name'], imgt_hc['V-D-J-REGION'])))
binding_table['VL_AA'] = binding_table['name'].map(dict(zip(imgt_lc['clone_name'], imgt_lc['V-J-REGION'])))

In [65]:
# This is cutoff they use in paper
binders = binding_table[binding_table['ELISA EC50 [µg/mL]'] != '>1']

In [66]:
# Took E1/E2 from here:
# https://www.ncbi.nlm.nih.gov/protein/1479555230
# Was linked in paper
e1e2 = "rnasgv yhvtndcpns sivyeaedli lhapgcvpcv rqgnvsrcwvqitptlsaps lgavtaplrr avdylaggaa lcsalyvgda cgavflvgqm ftysprrhnvvqdcncsiys ghitghrmaw dmmmnwsptt alvmaqllri pqvvidiiag ahwgvlfaaayyasaanwak vvlvlflfag vdastrtvgg saaqgargla slftpgpqqn lqlintngswhinrtalncn dslqtgfvag llyyhkfnst gcpqrmascr plaafdqgwg tisyaavsgpsddkpycwhy pprpcgivpa rgvcgpvycf tpspvvvgtt drkgnptysw genetdifllnntrpptgnw fgctwmnstg fvktcgappc nlgptgnnsl kcptdcfrkh pdatytkcgsgpwltprclv hypyrlwhyp ctlnytifkv rmyigglehr levacnwtrg ercdledrdraelspllhtt tqwailpcsf tptpalstgl ihlhqnivdt qylyglsssi vswavkweyivlaflllada"
e1e2 = e1e2.replace(' ', '').upper()
print(len(e1e2))

536


In [72]:
binders['antigen_seq'] = e1e2
binders['antigen_name'] = 'HCV E1E2'

C:\Users\perry\AppData\Local\Temp\ipykernel_23996\2545317518.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  binders['antigen_seq'] = e1e2
C:\Users\perry\AppData\Local\Temp\ipykernel_23996\2545317518.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  binders['antigen_name'] = 'HCV E1E2'


In [74]:
binders.to_csv('HCV_weber_seqs_v2_24-03-07.csv')